# 1. Libraries Import

In [ ]:
# ========================================================
# = Libraries import
# ========================================================

# from collections import defaultdict
import numpy as np
import boto3
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import datetime
from datetime import timedelta
# from pandas.tseries.offsets import DateOffset

#import os

#from timezonefinder import TimezoneFinder
import pytz

#import math

import pvlib
from pvlib import irradiance
#from pvlib import location

# import pickle # dump variables

#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#import plotly.graph_objects as go

# 2. AWS credentials

In [ ]:
# ========================================================
# = AWS Credentials
# ========================================================

PROD_AWS_PROFILE = "gsesami-prod"
AWS_REGION = "ap-southeast-2"

prod_session = boto3.session.Session(profile_name=PROD_AWS_PROFILE)

prod_client = prod_session.client(
    "timestream-query", region_name=AWS_REGION)

In [ ]:
# Time period
time_start = '2020-01-01'

# Setting date_end to today
today = datetime.date.today().strftime('%Y-%m-%d')
time_end = today

In [ ]:
# ========================================================
# = Thresholds
# ========================================================

# Cloudiness
# Define the threshold for low cloudiness days:
threshold_low_cloudiness = 80

# 3. Querying TimeStream

## 3.1. Monitor ID, Site ID, and Labelled Faults

In [ ]:
# Reading all sites
sites_list = pd.read_csv('./input_data/Site_List.csv')
# Reading all monitors
monitors_list = pd.read_csv('./input_data/Monitors_List.csv')

In [ ]:
# Function to read from the labelled dataset
def get_site_id_full(df, sequence):
    site_id_full = df['siteId'].loc[sequence]
    return site_id_full

def get_site_id(site_id_full):
    site_id = site_id_full.removeprefix('SITE|')
    return site_id

def get_mid_full(df, sequence):
    MID_full = df['source'].loc[sequence]
    return MID_full

def get_mid_id(MID_full):
    MID = MID_full.removeprefix('MNTR|')
    return MID

# =======================
# = Aggregate Function
# =======================

def get_mid_info(df, sequence):
    site_id_full = get_site_id_full(df, sequence)
    site_id = get_site_id(site_id_full)
    MID_full = get_mid_full(df, sequence)
    MID = get_mid_id(MID_full)

    return site_id_full, site_id, MID_full, MID

In [ ]:
def get_site_id(site_id_full):
    site_id = site_id_full.removeprefix('SITE|')
    return site_id

# =======================
# = Aggregate Function
# =======================

def get_site_info(df, sequence):
    site_id_full = get_site_full(df, sequence)
    site_id = get_site_id(site_id_full)

    return site_id_full, site_id

## 3.2. Getting Timezone

In [ ]:
def get_timezones(site_id_full, sites_list, time_start, time_end):
    # Checking timezone, using Sydney as a backup
    timezone_value = 'Australia/Sydney'
    timezone_value = sites_list[sites_list['source'] == site_id_full].iloc[0]['timezone']

    time_starttz = pytz.timezone('UTC').localize(datetime.datetime.strptime(time_start, '%Y-%m-%d'))
    time_endtz = pytz.timezone('UTC').localize(datetime.datetime.strptime(time_end, '%Y-%m-%d'))

    print(f'This monitor is located at: {timezone_value}')
    
    return timezone_value, time_starttz, time_endtz

## 3.3. Getting Monitor Metadata

In [ ]:
def fetch_monitor_metadata(monitors_list):

    print('-- Fetching Monitor Metadata')

    result = {}

    # Making sure the monitor list is of type string:
    monitors_list['source'] = monitors_list['source'].astype(str)

    # filtering the dataframe based on the monitor in question:
    monitor_row = monitors_list.loc[monitors_list['source'] == MID_full]

    # If the monitor ID exists, isolate each useful variable:
    if not monitor_row.empty:
        # latitude
        ## Notice that the data output here for latitude is really weird,
        ## Have to fix it a bit:
        result['mid_latitude'] = float(monitor_row['latitude'].values[0])
        # longitude:
        result['mid_longitude'] = float(monitor_row['longitude'].values[0])
        # loss:
        result['mid_loss'] = float( 1 - monitor_row['loss'].values[0] )
        # manufacturer api:
        result['mid_manufacturerApi'] = monitor_row['manufacturerApi'].values[0] 
        # pvSize:
        result['mid_pvsizewatt'] = monitor_row['pvSizeWatt'].values[0]
        # tilt:
        result['mid_tilt'] = float(monitor_row['tilt'].values[0])
        # weatherstationid:
        result['mid_weatherStationId'] = monitor_row['weatherStationId'].values[0]
        # Azymuth:
        result['mid_azimuth'] = float(monitor_row['azimuth'].values[0])

        print(f'Fetching Metadata')
    else:
        print(f"No data found for the Monitor ID {MID}")

    return result

## 3.3 Helper functions to read metrics

In [ ]:
def read_metric(time_start, time_end, measure_name, MID):
    """
    read raw data from the AWS database
    :param time_start: time start, e.g., '2022-10-02'
    :param time_end: time end, e.g., '2023-04-05'
    :param measure_name: measurement metric, e.g.,'Gen.W'
    :param MID: monitor id
    :return:
    """
    timeid = []
    data_values = []
    ##----------------- read the Performance  --------------##
    query = """SELECT time, measure_value::bigint
                    FROM "DiagnoProd"."DiagnoProd"
                    WHERE measure_name = '""" + measure_name + """'
                    AND MID = '""" + MID + """'
                    AND time BETWEEN '""" + time_start + """'
                    AND '""" + time_end + """' """

    client = prod_client
    paginator = client.get_paginator("query")
    page_iterator = paginator.paginate(QueryString=query,)
    i = 1
    for page in page_iterator:
        # print(page)
        try:
            timeid_page = [f[0]['ScalarValue'] for f in pd.DataFrame(page["Rows"])['Data']]
            data_values_page = [f[1]['ScalarValue'] for f in pd.DataFrame(page["Rows"])['Data']]
            timeid = timeid + timeid_page
            data_values = data_values + data_values_page
        except KeyError:
            print('Page {%d} has no data available:'%i)
        i = i+1
    return timeid, data_values

In [ ]:
def build_dataframe(timeid, measure_name, data_values, timezone_value):
    """
    change the time zone
    :param timeid: time read from the AWS
    :param measure_name:
    :param data_values: value read from the ASW
    :param timezone_value: time zone
    :return:
    """
    timeid = pd.to_datetime(timeid)
    if timeid.tzinfo is None:
        print('this is not tz-aware')
        if timezone_value is not None:
            timeid = timeid.tz_localize('UTC').tz_convert(timezone_value)
        else:
            print('no timezone in the table')
            timeid = timeid.tz_localize('UTC').tz_convert('Australia/Sydney')
    else:
        print('this is tz-aware')
    data = pd.DataFrame(data={'time':timeid, measure_name: data_values})
    data.sort_values('time', inplace=True)
    # data.set_index('time', inplace=True)
    data[measure_name] = data[measure_name].astype(float)
    return data

In [ ]:
def change_tz(timeid):
    # print('rawtimeid:', timeid)
    tzinfo_str = timeid[0].tzinfo
    hour_offset = tzinfo_str.utcoffset(datetime.datetime(2022,1,1))
    hms = str(hour_offset).split(':')
    time_modified = timeid + datetime.timedelta(hours=int(hms[0]), minutes=int(hms[1]), seconds=int(hms[2]))
    time_utc = time_modified.dt.tz_convert('UTC')
    # print('modified:', time_utc)
    return time_utc

In [ ]:
def setup_dataframe(time_starttz, time_endtz):
    print('-- Starting Dataframe Setup')

    # Setting up a 5-min dataframe
    time_index5min = pd.date_range(start=time_starttz, end=time_endtz, freq='5min').tz_convert('UTC')
    df_5min = pd.DataFrame(index=np.arange(len(time_index5min)))
    df_5min['time'] = time_index5min
    return df_5min

In [ ]:
# ==============================================#
# = Reading P(AC) total from AWS TimeStream     #
# = Metric is Gen.W                             #
# ==============================================#

def get_genW_dataframe(df_5min, time_start, time_end, measure_name, MID, timezone_value):
    print(f'Reading from DyanmoDB and building Gen.W dataframe')
    timeid, data_values = read_metric(time_start, time_end, measure_name, MID)
    df_genW = build_dataframe(timeid, measure_name, data_values, timezone_value)

    # =====================================================#
    # = Adjusting the df_genW to have 5 minutes increments #
    # =====================================================#

    # Convert the 'time' column in df_5min to timezone
    df_5min['time'] = df_5min['time'].dt.tz_convert(timezone_value)
    df_genW = pd.merge_asof(df_5min, df_genW, on="time")

    # Getting the first valid index:
    first_valid_index = df_genW['Gen.W'].first_valid_index()
    df_genW = df_genW[first_valid_index:].reset_index(drop=True)

    # making sure there's no negative generation
    ## I'll be saving a copy of the original Gen.W to include an analysis of negative generation afterwards
    df_genW['original_genW'] = df_genW['Gen.W']

    # Now clipping
    df_genW['Gen.W'] = df_genW['Gen.W'].clip(lower=0)
    
    return df_genW

## 3.4. Helper functions to get low-cloudiness days

In [ ]:
def read_metric_site(date_start, date_end, measure_name, site_id):
    timeid = []
    data_values = []
    ##----------------- read the Performance  --------------##
    query = """SELECT date, max_by(measure_value::double, time) as prod_val
                FROM "DiagnoProd"."DiagnoProd"
                WHERE measure_name = '""" + measure_name + """'
                AND siteId = '""" + site_id + """'
                AND date BETWEEN '""" + date_start + """'
                AND '""" + date_end + """'
                GROUP BY date
                ORDER BY date """
    
    client = prod_client
    paginator = client.get_paginator("query")
    page_iterator = paginator.paginate(QueryString=query,)
    i = 1
    for page in page_iterator:
        # print(page)
        try:
            timeid_page = [f[0]['ScalarValue'] for f in pd.DataFrame(page["Rows"])['Data']]
            data_values_page = [f[1]['ScalarValue'] for f in pd.DataFrame(page["Rows"])['Data']]
            timeid = timeid + timeid_page
            data_values = data_values + data_values_page
        except KeyError:
            print('Page {%d} has no data available:'%i)
        i = i+1
    return timeid, data_values

In [ ]:
def build_dataframe_site(timeid, measure_name, data_values):
    # ============== Check if there is data available for the pv system =============
    if len(timeid)!=0:
        timeid = pd.to_datetime(timeid)
        if timeid.tzinfo is None:
            print('this is not tz-aware')
            if timezone_value is not None:
                timeid = timeid.tz_localize('UTC').tz_convert(timezone_value)
                # timeid = timeid.tz_localize(timezone_list[i])
            else:
                print('no timezone in the table')
                timeid = timeid.tz_localize('UTC').tz_convert('Australia/Sydney')
                # timeid = timeid.tz_localize('Australia/Sydney')
        else:
            print('this is tz-aware')
        
        timesort = timeid.sort_values()
        data = pd.DataFrame(data={'time':timeid, measure_name: data_values})
        data.sort_values('time', inplace=True)
        data.set_index('time', inplace=True)
        data[measure_name] = data[measure_name].astype(float)
    else:
        data = pd.DataFrame(data_values, index=timeid, columns=[measure_name])
    
    return data

In [ ]:
# ==================================
# = Merging clear skies and expected
# ==================================

def merge_clear_expe(df1, df2):
    df_merged = df1.join(df2)
    df_merged['expected_over_clear'] =  (df_merged['Irrad.kWh.m2.Daily'] / df_merged['EnergyYield.kWh.Daily'] * 100).round(0)
    df_merged['date'] =  df_merged.index
    return df_merged

In [ ]:
def fetch_and_process_cloudiness_data(time_start, time_end, site_id, threshold_low_cloudiness):
        
        # ====================================================#
        # = Reading EnergyYield.kWh.Daily from AWS TimeStream #
        # ====================================================#

        print(f'Setting up low-cloudiness days')

        print(f'-- Reading EnergyYield')
        measure_name = 'EnergyYield.kWh.Daily'
        timeid, data_values = read_metric_site(time_start, time_end, measure_name, site_id)
        df_clear = build_dataframe_site(timeid, measure_name, data_values)

        # =================================================#
        # = Reading Irrad.kWh.m2.Daily from AWS TimeStream #
        # =================================================#
        
        print(f'-- Reading Irrad')
        measure_name = 'Irrad.kWh.m2.Daily'
        timeid, data_values = read_metric_site(time_start, time_end, measure_name, site_id)
        df_expected = build_dataframe_site(timeid, measure_name, data_values)

        # Fixing it as a float:
        df_expected['Irrad.kWh.m2.Daily'] = df_expected['Irrad.kWh.m2.Daily'].astype(float)

        # ===================================================#
        # = Reading Production.kWh.Daily from AWS TimeStream #
        # ===================================================#

        print(f'-- Reading Production Daily')
        measure_name = 'Production.kWh.Daily'
        timeid, data_values = read_metric_site(time_start, time_end, measure_name, site_id)
        df_production = build_dataframe_site(timeid, measure_name, data_values)

        # Fixing it as float
        df_production['Production.kWh.Daily'] = df_production['Production.kWh.Daily'].astype(float)

        # Merging clear and expected:
        df_merged = df_clear.join(df_expected)

        # Merging (clear and expected) and production
        df_merged = df_merged.join(df_production)

        # Getting the performance ratio:
        df_merged['expected_over_clear'] =  (df_merged['Irrad.kWh.m2.Daily'] / df_merged['EnergyYield.kWh.Daily'] * 100).round(0)

        # Getting this extra column flor plotting:
        df_merged['date'] =  df_merged.index.date

        # ====================================================================================#
        # = Checking values above a certain threshold when comparing clear skies and expected #
        # ====================================================================================#

        print(f'-- Checking Cloudy Days')

        # Make it low_cloudiness aware:
        df_merged.loc[df_merged['expected_over_clear'] >= threshold_low_cloudiness, 'is_low_cloudiness_day'] = True 
        df_merged.loc[df_merged['expected_over_clear'] < threshold_low_cloudiness, 'is_low_cloudiness_day'] = False

        return df_merged

# 4. Data from PVLib

Reference:

https://pvlib-python.readthedocs.io/en/stable/user_guide/clearsky.html

In [ ]:
def get_irradiance(loc, times, tilt, surface_azimuth):
    # Generate clearsky data using the Ineichen model, which is the default
    # The get_clearsky method returns a dataframe with values for GHI, DNI,and DHI
    clearsky = loc.get_clearsky(times)
    # Get solar azimuth and zenith to pass to the transposition function
    solar_position = loc.get_solarposition(times=times)
    # Use the get_total_irradiance function to transpose the GHI to POA
    POA_irradiance = irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=surface_azimuth,
        dni=clearsky['dni'],
        ghi=clearsky['ghi'],
        dhi=clearsky['dhi'],
        solar_zenith=solar_position['apparent_zenith'],
        solar_azimuth=solar_position['azimuth'])
    # Return DataFrame with only GHI and POA
    return pd.DataFrame({'GHI': clearsky['ghi'],
                         'POA': POA_irradiance['poa_global']})

In [ ]:
def compute_sun_and_irradiance(df_genW, timezone_value, monitor_metadata, time_start, time_end):
        
        # Every day only has 1 sunrise and 1 sunset, so we can go with uique days:
        unique_dates = df_genW['time'].dt.date.unique()

        # initialising:
        sun_info = {}
        # iterating over my array of uniquedates:
        for date in unique_dates:
            # pvlib expects a localised value, not an iterable list, so I'm wrapping my date timestamp:
            localized_date = pd.DatetimeIndex([pd.Timestamp(date).tz_localize(timezone_value)]) 
            # getting the results, which is a 1 row dataframe with 3 columns:
            sun_results = pvlib.solarposition.sun_rise_set_transit_spa(localized_date, monitor_metadata['mid_latitude'], monitor_metadata['mid_longitude'])
            sun_info[date] = sun_results.values[0] 

        # Convert dictionary to a DataFrame:
        sun_info_df = pd.DataFrame.from_dict(sun_info, orient='index', columns=['sunrise', 'sunset', 'transit'])

        # creating a date column for the merge:
        df_genW['date'] = df_genW['time'].dt.date

        # Reset index in sun_info_df and rename the index column as 'date'
        sun_info_df = sun_info_df.reset_index().rename(columns={'index':'date'})

        # Convert the 'date' column to datetime in both dataframes
        df_genW['date'] = pd.to_datetime(df_genW['date'])
        sun_info_df['date'] = pd.to_datetime(sun_info_df['date'])

        # Merge df_genW with sun_info_df
        df_genW = pd.merge(df_genW, sun_info_df, on='date', how='left')

        # Convert 'time' column to datetime, to be sure:
        df_genW['time'] = pd.to_datetime(df_genW['time'])

        # Create a copy of 'time' column before setting it as index, I'll need this for the labelling
        df_genW['time_copy'] = df_genW['time']

        # Set 'time' column as index
        df_genW.set_index('time', inplace=True)

        # Running pvlib to get GHI:
        location = pvlib.location.Location(monitor_metadata['mid_latitude'], monitor_metadata['mid_longitude'], tz=timezone_value)
        df_genW['clear_sky_ghi'] = location.get_clearsky(df_genW.index, model='ineichen')['ghi']

        # Renaming time_copy to time again:
        df_genW.rename(columns={'time_copy': 'time'}, inplace=True)

        # ================================================================#
        # = Getting POA and GHI using PVLib, and setting up the dataframe #
        # ================================================================#

        # Setting up a dataframe with local info:
        time_index5min_local = pd.date_range(start=pd.to_datetime(time_start).tz_localize(timezone_value), end=pd.to_datetime(time_end).tz_localize(timezone_value), freq='5min')

        # Getting the location
        loc = pvlib.location.Location(monitor_metadata['mid_latitude'], monitor_metadata['mid_longitude'], tz=timezone_value)
        pvlib_irr_pre = get_irradiance(loc, time_index5min_local, monitor_metadata['mid_tilt'], monitor_metadata['mid_azimuth'])

        # Correcting for size and loss
        pvlib_irr_pre['POA'] = pvlib_irr_pre['POA'] * monitor_metadata['mid_pvsizewatt'] * monitor_metadata['mid_loss'] / 1000

        # Merging the dataframes
        merged_df = pd.merge(df_genW, pvlib_irr_pre, left_index=True, right_index=True)

        # Assign the column
        merged_df['theoretical_clear-sky_generation.W'] = merged_df['POA']

        return merged_df

# 5. Transformations for recurring fault analysis

In [ ]:
def resample_and_threshold(df_genW):
    df_genW_resample = df_genW.copy()  # create a copy of the original dataframe

    df_genW_resample['time'] = pd.to_datetime(df_genW_resample['time']) # convert time to datetime if it's not
    df_genW_resample.set_index('time', inplace=True) # set time as index

    # Create new df with resampling
    df_genW_resample = df_genW_resample.resample('H').mean()

    return df_genW_resample

In [ ]:
# Note that cadence_of_observation is in minutes.

def shift_timeseries(df,cadence_of_observation = 5):
    # Finding the first non-zero generation timestamp for each DataFrame
    ## Measured
    start_genW = df[df['Gen.W'] != 0]['time'].iloc[0]
    ## Clear Skies
    start_theoretical = df[df['theoretical_clear-sky_generation.W'] != 0]['time'].iloc[0]

    # Compute time difference in hours
    time_diff = (start_theoretical - start_genW).total_seconds() / 3600

    # Converting timedelta into minutes
    shift_minutes = timedelta(hours=time_diff).seconds // 60

    print(f'shifting the timeseries by {shift_minutes} minutes')

    shift_periods = int(shift_minutes / cadence_of_observation)

    # Shift 'Gen.W' column in df_genW_copy
    df['Gen.W'] = df['Gen.W'].shift(shift_periods)

    # Fill NaN values (if any) with 0 after shifting
    df['Gen.W'].fillna(0, inplace=True)
    return df

In [ ]:
# Function to normalize columns based on the maximum value of theoretical_clear-sky_generation.W for each day
def normalize_daywise(group):

    # CUrrently hardcoded Gen.W -> Make sure to change this to a if stament to encompass the cases for when
    # On a day Gen.W has a higher max and on a day Theoritical has a higher max

    max_val = max(group['Gen.W'].max(), group['theoretical_clear-sky_generation.W'].max())

    # Avoiding division by zero and infinite results tehreafter:
    max_val = max_val if max_val != 0 else 1
    
    group['theoretical_clear-sky_generation.W_normalized'] = group['theoretical_clear-sky_generation.W'] / max_val
    
    group['Gen.W_normalized'] = group['Gen.W'] / max_val
    return group

In [ ]:
def stretch_theoretical(df):

    # I'm initialising an empty dict so I can keep track of the stretch factors:
    stretch_factors = {}

    # Threshold for theoretical value below which we won't compute the ratio
    ## The threshold helps in avoiding unrealistically high stretch factors caused by near-zero values in the denominator.
    
    ## Currently using 0.7, as 70% of the main normalised value (1)
    THRESHOLD = 0.7

    # Function to compute and adjust the theoretical curve for each day's group
    def compute_stretch(group):

        # Check if the input is a DataFrame and if not, return as is
        ## I've been getting some weird AttributeErrors here
        if not isinstance(group, pd.DataFrame):
            return group

        # getting the date from each row, so I can keep track of stretch factors
        current_date = group.index[0].date()

        # Compute ratio where theoretical value is above the threshold
        ## I want to make sure that I'm not dividing by tiny values and getting crazy spikes
        valid_idxs = group['theoretical_clear-sky_generation.W_normalized'] > THRESHOLD
        ratios = (group['Gen.W_normalized'] / group['theoretical_clear-sky_generation.W_normalized'])[valid_idxs]
        
        # Use maximum ratio as stretch factor
        stretch_factor = ratios.max()
        
        # If the stretch factor is NaN or zero (no valid ratios), set it to 1 to avoid NaN results
        # This will keep the value unchanged
        stretch_factor = 1 if pd.isna(stretch_factor) or stretch_factor == 0 else stretch_factor

        # Storing each stretch factor so I can keep track of it:
        stretch_factors[current_date] = stretch_factor

        # Multiply each value in the theoretical column by the stretched factor
        group['stretched_theoretical'] = group['theoretical_clear-sky_generation.W_normalized'] * stretch_factor
        return group

    # I'm grouping the datasframe by DAY and applying that function to it
    df_stretched = df.groupby(df.index.map(lambda x: x.date())).apply(compute_stretch)

    # Convert the stretch_factors dictionary to a DataFrame for better vis
    stretch_factors_df = pd.DataFrame(list(stretch_factors.items()), columns=['date', 'stretch_factor'])
    # Convert 'date' column to datetime and set as index
    stretch_factors_df['date'] = pd.to_datetime(stretch_factors_df['date'])
    stretch_factors_df.set_index('date', inplace=True)

    return df_stretched, stretch_factors_df


In [ ]:
def get_deltas(df_genW_resample, df_stretched):
        
    # =================#
    # = Getting Deltas #
    # =================#

    # Getting deltas
    df_genW_resample['time'] = df_genW_resample.index
    df_stretched['delta-clear-gen'] = df_stretched['Gen.W_normalized'] - df_stretched['stretched_theoretical']

    # simplifying
    df_MA = df_stretched.copy(deep=True)
    df_MA = df_MA.loc[:,['time','Gen.W','theoretical_clear-sky_generation.W','delta-clear-gen','Gen.W_normalized','stretched_theoretical']]
    df_MA = df_MA.resample('H').sum()
    df_MA['timestamp'] = df_MA.index
    df_MA = df_MA.rename(columns={"delta-clear-gen": "value"})
    
    # inverting
    df_MA_inverted = df_MA.copy(deep=True)
    df_MA_inverted['value'] = df_MA['value'] * (-1)

    return df_MA_inverted

In [ ]:
def merge_on_low_cloudiness(df_MA_inverted, df_site):
    # Convert 'timestamp' column in df_test to datetime if it's not
    df_MA_inverted['timestamp'] = pd.to_datetime(df_MA_inverted['timestamp'])

    # Create a new column in df_test with just the date (no time info)
    df_MA_inverted['date_only'] = df_MA_inverted['timestamp'].dt.date

    # Convert index 'time' to datetime and create a new column 'time' in df_site if it's not
    df_site['time'] = pd.to_datetime(df_site.index)

    # Create a new column in df_site with just the date (no time info)
    df_site['date_only'] = df_site['time'].dt.date

    # Merge df_site and df_test on the date_only column
    df_with_cloud = df_MA_inverted.merge(df_site[['date_only', 'is_low_cloudiness_day']], on='date_only', how='left')

    # If you want to drop the 'date_only' column after the merge:
    df_with_cloud = df_with_cloud.drop(columns=['date_only'])

    # Create a copy of 'timestamp' column
    df_with_cloud['timestamp_copy'] = df_with_cloud['timestamp']

    # Set 'timestamp' column as index
    df_with_cloud.set_index('timestamp', inplace=True)

    # Rename the 'timestamp_copy' back to 'timestamp'
    df_with_cloud.rename(columns={'timestamp_copy': 'timestamp'}, inplace=True)

    # Deep copy
    df_MA_inverted = df_with_cloud.copy(deep=True)

    return df_MA_inverted

In [ ]:
def limit_to_threshold(df, threshold_percentage=0.2):
    # Make sure index is a DateTimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # Get all unique dates in the df
    unique_dates = df.index.normalize().unique()

    # Iterate over each unique day in the df
    for date in unique_dates:
        # Convert to string for indexing
        str_date = date.strftime('%Y-%m-%d')

        # Get the max 'value' for the day
        max_value = df.loc[str_date, 'stretched_theoretical'].max()

        # Multiply the max 'value' by the threshold
        threshold = max_value * threshold_percentage

        # Find 'Gen.W' values that are below the threshold and replace them with 0
        df.loc[(df.index.normalize() == date) & (df['value'] < threshold), 'value'] = 0

    return df

In [ ]:
def mark_recurrent_underperformance(df, threshold_of_recurrence=3, value_threshold_percentage=20):
    
    if df.index.name != 'timestamp':
        df.set_index('timestamp', inplace=True)
    
    df['hour'] = df.index.hour
    df['underperformance'] = df['value'] > 0  # Assuming 'value' > 0 means underperformance
    df['Recurring Underperformance'] = False
    df['count_for_hour_of_day'] = 0
    
    # Filter only rows where is_low_cloudiness_day is True
    low_cloudiness_df = df[df['is_low_cloudiness_day'].fillna(False)]
    
    for hour in range(24):  # Looping through all hours of the day
        hourly_df = low_cloudiness_df[low_cloudiness_df['hour'] == hour].copy()
        
        count = 0
        potential_indices = []  # To keep track of the potential recurrent underperformances
        
        for idx, row in hourly_df.iterrows():
            
            if row['underperformance'] and row['value'] != 0:
                
                # For the first encounter with a fault
                if not potential_indices:  # List is empty, initialize
                    count = 1
                    potential_indices = [idx]
                    df.loc[idx, 'count_for_hour_of_day'] = count
                    reference_value = row['value']
                    
                else:
                    # Compute the percentage difference
                    reference_value = hourly_df.loc[potential_indices[0], 'value']
                    percentage_diff = abs((row['value'] - reference_value) / reference_value) * 100
                    
                    # Check if the value is within the threshold
                    if percentage_diff <= value_threshold_percentage:
                        count += 1
                        df.loc[idx, 'count_for_hour_of_day'] = count
                        potential_indices.append(idx)  # Store this index
                    else:
                        # Reset count and start tracking from the current index
                        count = 1  
                        df.loc[idx, 'count_for_hour_of_day'] = count
                        potential_indices = [idx]  # Reset potential_indices list

                # Check if count reaches the threshold
                if count >= threshold_of_recurrence:
                    # Set 'is_recurrent_underperformance' to True for all potential indices
                    df.loc[potential_indices, 'Recurring Underperformance'] = True
                
            else:
                # Reset count and clear list
                count = 0  
                potential_indices = []  
                
    return df

# 6. Functions to label Level 1 faults based on raw signal

## 6.1. Generation Tripping

In [ ]:
def detect_solar_tripping(df, trip_threshold=3):
    # Ensure index is in the correct format
    df['time'] = pd.to_datetime(df['time'])
    df['date_from_time'] = df['time'].dt.date
    
    # Create a boolean mask where generation is zero
    df['is_zero_gen'] = df['Gen.W'] == 0

    # Restrict to periods between sunrise and sunset
    df['is_zero_gen'] = df['is_zero_gen'] & \
                                        (((df.index.hour * 60) + df.index.minute)>= df['sun_thre_start']) & (((df.index.hour * 60) + df.index.minute) <= df['sun_thre_end'])

    # Identifying instances where generation goes to zero and then back up

        ## First identify where is_zero_gen changes from False to True or True to False
    df['diff_is_zero_gen'] = df['is_zero_gen'].astype(int).diff()

        ## Then isolate instances where the generation is greater than 0
    df['positive_gen'] = df['Gen.W'] > 0

        ## Finally, combining both conditions
    df['is_tripping'] = (df['diff_is_zero_gen'] != 0) & df['positive_gen']

    # Get daily counts of tripping
    daily_tripping_counts = df.resample('D')['is_tripping'].sum()

    # Identify the dates where tripping counts cross the threshold
    threshold_dates = daily_tripping_counts[daily_tripping_counts >= trip_threshold].index.date

    # Convert df.index.date to a pandas Series
    df_dates = pd.Series(df.index.date)
    
    # Update 'fault_tripping' column based on threshold_dates
    df['fault_tripping'] = df['date_from_time'].isin(threshold_dates)

    # I'm filling NaN values with False
    df['fault_tripping'] = df['fault_tripping'].fillna(False)
    
    return df

## 6.2. Non-zero generation tripping

In [ ]:
def detect_nonZero_tripping(df, threshold_nonZero_tripping=0.8):
    # Initiatlising the column as False
    df['nonZero_tripping'] = 0  
    df['fault_nonZero_tripping'] = False  

    # looping through and getting individual drops:
    for i in range(1, len(df)):
        # Get a ratio from measurements right after another
        ratio = df['Gen.W'].iloc[i] / df['Gen.W'].iloc[i-1]

        # If a drop is too significant, consider it a non-zero tripping:
        if ratio < threshold_nonZero_tripping:
            df['nonZero_tripping'].iloc[i] = 1

    # Summing up all drops on a given day:
    df['nonZero_tripping_sum'] = df['nonZero_tripping'].resample('D').transform('sum')

    # Marking a day that this happens at least thrice:
    df.loc[df['nonZero_tripping_sum'] >= 3, 'fault_nonZero_tripping'] = True
    
    return df

## 6.3. Generation Clipping - Normalised with PVSize

In [ ]:
def detect_solar_clipping(df, mid_pvsizewatt, threshold=0.001, min_duration='60min'):
    df = df.sort_index()

    # Calculate the rate of change of power
    # Currently using diff, should use derivative?
    
    df['power_diff_normalised'] = df['Gen.W'].diff()/mid_pvsizewatt

    # We only want to check clipping in between the thresholds, therefore we'll need to define the hour:
    # df['hour'] = df.index.hour

    # Identifying periods where:
    ##  the rate of change is near zero (potential clipping, considered by threshold), and 
    ## there more than 100 generation (sun_threshold), and
    ## is between sun_thre_start and sun_thre_end
    df['is_clipping'] = (np.abs(df['power_diff_normalised']) < threshold) & \
                        (df['Gen.W'] > 100) & \
                        (((df.index.hour * 60) + df.index.minute)>= df['sun_thre_start']) & (((df.index.hour * 60) + df.index.minute) <= df['sun_thre_end'])
    
    # Identify 'clipping periods', which are periods of clipping that last at least min_duration
    ## First we get EVERY occurrence of clipping, for every 5 min
    df['clipping_period'] = df['is_clipping'].diff().ne(0).cumsum()

    # Then, we'll calculate the duration of each 'clipping period'
    df['clipping_duration'] = df.groupby('clipping_period')['is_clipping'].transform('sum') * (df.index.to_series().diff().dt.total_seconds() / 60)

    # If duration is longer than our min_duration, we label that as clipping
    df['fault_clipping'] = df['is_clipping'] & (df['clipping_duration'] >= pd.Timedelta(min_duration).total_seconds() / 60)
    
    return df

## 6.4. Zero Generation

In [ ]:
def detect_zero_generation(df, min_duration='60min'):  
    # Creating a boolean mask where generation is zero
    # tbd -> optimise this with tripping
    df['is_zero_gen'] = df['Gen.W'] == 0

    # Restrict to periods between sunrise and sunset
    df['is_zero_gen'] = df['is_zero_gen'] & (((df.index.hour * 60) + df.index.minute)>= df['sun_thre_start']) & (((df.index.hour * 60) + df.index.minute) <= df['sun_thre_end'])

    '''   
        # Trying with zero_gen and ghi not zero:
        df['is_zero_gen'] = df['is_zero_gen'] & df['clear_sky_ghi'] > 0
    '''

    # Labeling periods of zero generation
    df['zero_gen_period'] = df['is_zero_gen'].diff().ne(0).cumsum()

    # Calculate the duration of each 'zero_gen_period'
    df['zero_gen_duration'] = df.groupby('zero_gen_period')['is_zero_gen'].transform('sum') * (df.index.to_series().diff().dt.total_seconds() / 60)

    # If duration is longer than our min_duration, we label that as zero_gen_period
    df['fault_zero_gen'] = df['is_zero_gen'] & (df['zero_gen_duration'] >= pd.Timedelta(min_duration).total_seconds() / 60)
    
    return df

## 6.5. Recurring underperformance

## 6.6. Night-time Generation

In [ ]:
def detect_nightTime_gen(df):
    # Create a boolean column for generation during night time
    df['temp_fault_nightTime_gen'] = (df['Gen.W'] > 0) & \
                                     ~((((df.index.hour * 60) + df.index.minute) >= df['sun_thre_start']) & \
                                       (((df.index.hour * 60) + df.index.minute) <= df['sun_thre_end']))

    # Create a new column to indicate if the nighttime generation occurred continuously for 1 hour (12 intervals)
    df['fault_nightTime_gen_1hr'] = df['temp_fault_nightTime_gen'].rolling(window=12, min_periods=12).sum() == 12

    # Drop the temporary column
    df.drop(columns=['temp_fault_nightTime_gen'], inplace=True)

    return df

## 6.7. Negative Generation

In [ ]:
# Note, need to use df_toCheck_negativeGen

def detect_negative_gen(df, mid_pvsizewatt):

    # Calculating the threshold (1% of mid_pvsizewatt)
    threshold_neg = 0.01 * mid_pvsizewatt

    # Creating a boolean to catch negative values
    df['fault_negative_gen'] = (df['original_genW'] < 0) & (abs(df['original_genW']) >= threshold_neg)

    return df

## 6.8. Excessive Generation

In [ ]:
def detect_excessive_gen(df, mid_pvsizewatt):

    # 100% on top of system size
    threshold_excessive = 2 * mid_pvsizewatt

    df['fault_excessive_gen'] = (df['Gen.W'] >= threshold_excessive)

    return df

## 6.9. No Data

In [ ]:
def detect_no_data(df):
    df['No Data'] = False
    if df['Gen.W'].isna().all():
        df['No Data'] = True
        
    return df

# 7. Label all

In [ ]:
def label_dataframe(df):
    # Extract hour from sunrise and sunset times

    # Extract time from sunrise and sunset times
    ## I was getting false positives since the sunset could be at 17:01, and by extracting the hour from the timestamp, I would get 17, and in that case I would have 12 possible slots to detect fault.
    ## I'm doing a 1 hour buffer for the start, and flooring the sunset
    df['index_minute_vale'] = ((df_genW.index.hour * 60) + df_genW.index.minute)
    df['sun_thre_start'] = (((df['sunrise'].dt.hour + 1) % 24) * 60) + df['sunrise'].dt.minute
    df['sun_thre_end'] = (df['sunset'].dt.hour * 60) + df['sunrise'].dt.minute
    
    df = detect_solar_tripping(df)
    df = detect_nonZero_tripping(df)
    df = detect_solar_clipping(df, monitor_metadata['mid_pvsizewatt'])
    df = detect_zero_generation(df)
    df = detect_negative_gen(df, monitor_metadata['mid_pvsizewatt'])
    df = detect_nightTime_gen(df)
    df = detect_excessive_gen(df, monitor_metadata['mid_pvsizewatt'])
    df = detect_no_data(df)
    
    return df

# 8. Cleaning up

In [ ]:
def find_faults(row):
    faults = []
    if row['fault_tripping']:
        faults.append('fault_tripping')
    if row['fault_nonZero_tripping']:
        faults.append('fault_nonZero_tripping')
    if row['fault_clipping']:
        faults.append('fault_clipping')
    if row['fault_zero_gen']:
        faults.append('fault_zero_gen')
    if row['fault_recurrent_underperformance']:
        faults.append('fault_recurrent_underperformance')
    if row['fault_negative_gen']:
        faults.append('Negative Generation')
    if row['fault_nightTime_gen_1hr']:
        faults.append('Night-Time Generation')
    if row['fault_excessive_gen']:
        faults.append('Excessive Generation')
    if row['No Data']:
        faults.append('No Data')
    return faults

# 9. Saving individual

In [ ]:
'''# To save it:
df_labelled.to_csv(f'./2B_monitor_results/individual_monitors/{MID}.csv')

print(f'Fault Detection saved for {MID}')'''

In [ ]:
'''df_per_day = df_labelled.copy()

df_per_day.index = df_per_day.index.date

def agg_faults_per_day(faults_list):
    unique_faults_on_that_day = list(set([fault for sublist in faults_list for fault in sublist]))
    return unique_faults_on_that_day

result_per_day = df_per_day.groupby(df_per_day.index).agg({'Gen.W': 'sum', 'faults': agg_faults_per_day})

result_per_day.to_csv(f'./2B_monitor_results/per_day/{MID}_per_day.csv')'''

# 10. Aggregate Analysis

In [ ]:
for i in range(len(monitors_list)):
    try:

        # =======================================================#
        #  1. Initialisation + Site and Monitor data collection  #
        # =======================================================#

        site_id_full, site_id, MID_full, MID = get_mid_info(monitors_list, i)

        print(f'Beginning Analysis for Monitor: {MID} under {site_id_full}')

        timezone_value, time_starttz, time_endtz = get_timezones(site_id_full, sites_list, time_start, time_end)

        monitor_metadata = fetch_monitor_metadata(monitors_list)

        # =========================================#
        #  2. Building a Gen.W dataframe from AWS  #
        # =========================================#

        df_5min = setup_dataframe(time_starttz, time_endtz)
        df_genW = get_genW_dataframe(df_5min, time_start, time_end, 'Gen.W', MID, timezone_value)

        # ===============================================================#
        # = Resampling a 1-hour dataframe for recurrent underperformance #
        # ===============================================================#

        # Creating a copy of the 'time' column
        df_genW['time_copy'] = df_genW['time']

        # Setting the 'time' column as the index
        df_genW = df_genW.set_index('time')

        # Resampling the dataframe hourly
        df_genW_resampled = df_genW.resample('H').mean()

        # Reseting the index
        df_genW_resampled.reset_index(level=0, inplace=True)
        df_genW.reset_index(level=0, inplace=True)

        # ================================#
        # = Information on low-cloudiness #
        # ================================#

        df_site = fetch_and_process_cloudiness_data(time_start, time_end, site_id, threshold_low_cloudiness)

        # =================================================#
        # = Using PVLib to get sunrise, sunset and transit #
        # =================================================#

        df_genW = compute_sun_and_irradiance(df_genW, timezone_value, monitor_metadata, time_start, time_end)

        # ====================================================================#
        # = Adding an hour resample to analyse the recurrent underperformance #
        # ====================================================================#

        df_genW_resample = df_genW.copy()  # create a copy of the original dataframe

        df_genW_resample['time'] = pd.to_datetime(df_genW_resample['time']) # convert time to datetime if it's not
        df_genW_resample.set_index('time', inplace=True) # set time as index

        # Create new df with resampling
        df_genW_resample = df_genW_resample.resample('H').mean()

        # ================================#
        # = Shifting resampled timeseries #
        # ================================#

        # Note that before shifting I want to apply a threshold on the aggregate hourly dataframe before shifting, to avoid situations in which there's more rows of measured than theoretical
        threshold_minimum_generation = df_genW_resample['Gen.W'].max() * 0.01
        threshold_minimum_theoretical = df_genW_resample['theoretical_clear-sky_generation.W'].max() * 0.01

        df_genW_resample.loc[df_genW_resample['Gen.W'] <  threshold_minimum_generation, 'Gen.W'] = 0
        df_genW_resample.loc[df_genW_resample['theoretical_clear-sky_generation.W'] <  threshold_minimum_theoretical, 'theoretical_clear-sky_generation.W'] = 0

        threshold_minimum_theoretical = df_genW_resample['theoretical_clear-sky_generation.W'].max() * 0.01
        df_genW_resample.loc[df_genW_resample['theoretical_clear-sky_generation.W'] <  threshold_minimum_theoretical, 'theoretical_clear-sky_generation.W'] = 0


        df_genW_resample['time'] = df_genW_resample.index
        df_genW_resample = shift_timeseries(df_genW_resample, cadence_of_observation=5)


        # ===================================#
        # = Normalising resampled timeseries #
        # ===================================#
        
        # Group by day and apply normalization
        df_normalized = df_genW_resample.groupby(df_genW_resample.index.date).apply(normalize_daywise)

        # ==================================#
        # = Stretching resampled timeseries #
        # ==================================#

        # Stretching
        df_stretched, stretch_factors_df = stretch_theoretical(df_normalized)

        # The values of 'theoretical_clear-sky_generation.W' should never be bigger than 'Gen.W'
        # For now, I'm using Gen.W as the cap
        df_stretched['Gen.W_normalized'] = np.where(df_stretched['Gen.W_normalized'] > df_stretched['stretched_theoretical'], df_stretched['stretched_theoretical'], df_stretched['Gen.W_normalized'])
    
        # =================#
        # = Getting Deltas #
        # =================#


        df_MA_inverted = get_deltas(df_genW_resample, df_stretched)

        # ==============================#
        # = Getting low-cloudiness info #
        # ==============================#

        df_MA_inverted = merge_on_low_cloudiness(df_MA_inverted, df_site)

        # ===============================#
        # = Limiting deltas to threshold #
        # ===============================#

        # Limiting to threshold
        df_MA_inverted = limit_to_threshold(df_MA_inverted)


        # ====================================#
        # = Maring recurrent underperformance #
        # ====================================#

        mark_recurrent_underperformance(df_MA_inverted)

        # ================#
        # = Labelling all #
        # ================#

        # I've already worked with df_MA_inverted, I'll just use it to label the original df_genW

        df_recurrent_und = df_MA_inverted.copy(deep=True)
        df_recurrent_und = df_recurrent_und[['timestamp','Recurring Underperformance']].rename(columns={'timestamp':'time'})

        # df_MA_inverted is a hourly dataframe, whereas df_genW has 5-minutes increments
        # df_MA_inverted['Recurring Underperformance'] is TRUE on an hourly basis:
        ## If it happens to be TRUE on a full hour, I'll propagate it to it's corresponding 5-minutes increments:
        # Using forward fill to achieve this:

        df_recurrent_und = df_recurrent_und.resample('5T').ffill()

        df_labelled = label_dataframe(df_genW)
        
        # ====================================#
        # = Labelling all #
        # ====================================#

        # I'll have to resample the df_MA_inverted to 5 minutes intervals
        # The only thing I really care about here is the 'Recurring Underperformance' column, so I'll use forward fill:
        # https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.ffill.html


        df_labelled['time'] = pd.to_datetime(df_labelled['time'])
        df_MA_inverted.index = pd.to_datetime(df_MA_inverted.index)

        df_recurrent_und = df_MA_inverted.copy(deep=True)
        df_recurrent_und = df_recurrent_und[['timestamp','Recurring Underperformance', 'is_low_cloudiness_day']].rename(columns={'timestamp':'time'})

        # Resample the df_MA_inverted to 5-minute intervals using forward filling
        df_recurrent_und = df_recurrent_und.resample('5T').ffill()

        # Merge the dataframes on the timestamp columns
        merged_df = pd.merge(df_labelled, df_recurrent_und[['Recurring Underperformance', 'is_low_cloudiness_day']], left_on='time', right_index=True, how='left')

        # Rename the Recurring Underperformance column
        merged_df.rename(columns={'Recurring Underperformance': 'fault_recurrent_underperformance'}, inplace=True)

        # Now merged_df contains the df_genW data with the added Recurring Underperformance column
        df_labelled = merged_df

        # If needed, you can fill any NaN values in the new column with a default value (e.g., False)
        df_labelled['fault_recurrent_underperformance'].fillna(False, inplace=True)

        # =======================================#
        # = Adding a column with array of faults #
        # =======================================#
        df_labelled['faults'] = df_labelled.apply(find_faults, axis=1)

        # =============================#
        # = Saving faults - every 5min #
        # =============================#

        # Saving the whole thing
        df_labelled.to_csv(f'./2B_monitor_results/individual_monitors/{MID}.csv')


        # Saving per day
        df_per_day = df_labelled.copy()

        df_per_day.index = df_per_day.index.date

        def agg_faults_per_day(faults_list):
            unique_faults_on_that_day = list(set([fault for sublist in faults_list for fault in sublist]))
            return unique_faults_on_that_day

        result_per_day = df_per_day.groupby(df_per_day.index).agg({'Gen.W': 'sum', 'faults': agg_faults_per_day})

        result_per_day.to_csv(f'./2B_monitor_results/per_day/{MID}_per_day.csv')

    except Exception as e:

        # =================#
        # = Error Handling #
        # =================#

        # This is a generic error handling, as specific errors are found, they can be placed under their respective place
        print(f"An unexpected error occurred: {e}")